Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import os


<li> Setting arbitrary passing_score = 70 for analysis going forward </li>
<li> Create data frames reading Schools data file </li>
 <li>   Rename the name column from 'name' to 'school_name' </li>
    

In [2]:
passing_score = 70
sch_csv_path = os.path.join("raw_data","schools_complete.csv")
schools_df = pd.read_csv(sch_csv_path)
# schools_df = schools_df.loc[schools_df["type"]=="District"]
schools_df = schools_df.reset_index(drop=True)
schools_df = schools_df.rename(columns={"name":"school_name"})
# schools_df


Compute number of schools into schools_count

In [3]:
schools_count = schools_df["school_name"].count()
# schools_count

Import students data file into students_df data frame
Rename school column as school_name

In [4]:
stu_csv_path = os.path.join("raw_data","students_complete.csv")
students_df = pd.read_csv(stu_csv_path)
students_df = students_df.rename(columns={"school":"school_name"})
# students_df.head()
# students_df.describe()

Merge school data frame with type='District' and students data

In [5]:
district_df = pd.merge(schools_df[schools_df.type == "District"],students_df,on="school_name")
# district_df.shape

Compute the number of schools into 'district_count'

In [6]:
district_count = len(district_df["school_name"].unique())
district_count

7

In [7]:
budget_total = district_df["budget"].sum()
# budget_total

In [8]:
avg_math_score = district_df["math_score"].mean()
# avg_math_score

In [9]:
avg_reading_score = district_df["reading_score"].mean()
# avg_reading_score

In [10]:
students_count = district_df["Student ID"].count()
# students_count

In [11]:
passing_math = district_df.loc[district_df["math_score"] >= passing_score].count() * 100/students_count
passing_math = passing_math["name"]

In [12]:
passing_reading = district_df.loc[district_df["reading_score"] >= passing_score].count() * 100/students_count
passing_reading = passing_reading["name"]

In [13]:
passing_list = [passing_math,passing_reading]
passing_rate = np.mean(passing_list)
# passing_rate

District Summary

Create a high level snapshot (in table form) of the district's key metrics, including:

Total Schools
Total Students
Total Budget
Average Math Score
Average Reading Score
% Passing Math
% Passing Reading
Overall Passing Rate (Average of the above two)

In [14]:
dict = {"Total Schools": district_count, "Total Students": students_count, "Total Budget": budget_total,
       "Average Math Score": avg_math_score, "Average Reading Score": avg_reading_score,
       "% Passing Math": passing_math, "% Passing Reading": passing_reading, "% Overall Passing Rate": passing_rate}
df = pd.DataFrame([dict], columns=dict.keys())
df.reset_index(drop=True)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,7,26976,70439053973,76.987026,80.962485,66.518387,80.905249,73.711818


create a school and students data from including all the data -- both District and Charter

In [15]:
schoolstu_df = pd.merge(schools_df,students_df,on="school_name")
school_sum = schoolstu_df.groupby(["school_name","type","name"])

In [16]:
# school_sum_no_students = district_df.groupby(["school_name"])
# school_sum_no_students['reading_score'].filter(lambda x: x > 70)

In [17]:
# t_school_math_sum = pd.DataFrame(district_df.groupby(["school_name"])["math_score"].mean().reset_index())
# t_school_math_sum = pd.DataFrame(district_df.groupby(["school_name"])(["math_score"]).mean().reset_index())
# t_school_math_sum


# pd.DataFrame(district_df.groupby(["school_name","grade"])["math_score"].mean().reset_index())

# t_school_math_pass = school_sum[school_sum["math_score"]> passing_score]

Compute the total of math_score greater than the passing score by each school.

In [18]:
school_sum_mp = school_sum.apply(lambda x: x[x["math_score"]>passing_score])
# school_sum_mp["school_name"].value_counts()
school_sum_mp = pd.DataFrame(school_sum_mp["school_name"].value_counts().reset_index())
school_sum_mp.columns = ["school_name", "math_pass"]
# school_sum_bySchool = pd.DataFrame(school_sum["math_score"].sum())
# school_sum_bySchool
# math = school_sum_bySchool.loc[school_sum_bySchool["math_score"]>=passing_score]
# math.value_count()

In [19]:
# school_sum_mp

Compute the total of math_score by each school

In [20]:
school_sum_sCounts = school_sum.apply(lambda x: x[x["math_score"]>=0])
school_sum_sCounts = pd.DataFrame(school_sum_sCounts["school_name"].value_counts().reset_index())
school_sum_sCounts.columns = ["school_name", "students_count"]
school_sum_sCounts

,school_name,students_count
0,Bailey High School,4976
1,Johnson High School,4761
2,Hernandez High School,4635
3,Rodriguez High School,3999
4,Figueroa High School,2949
5,Huang High School,2917
6,Ford High School,2739
7,Wilson High School,2283
8,Cabrera High School,1858
9,Wright High School,1800


Merge school summary table with total of math_score by school

In [21]:
school_sum_all = pd.merge(school_sum_sCounts,school_sum_mp,on="school_name")
# type(school_sum_all)

Compute the total reading_score greater then the passing score by each school

In [22]:
school_sum_rp = school_sum.apply(lambda x: x[x["reading_score"]>passing_score])
# school_sum_mp["school_name"].value_counts()
school_sum_rp = pd.DataFrame(school_sum_rp["school_name"].value_counts().reset_index())
school_sum_rp.columns = ["school_name", "reading_pass"]
school_sum_rp

,school_name,reading_pass
0,Bailey High School,3946
1,Johnson High School,3727
2,Hernandez High School,3624
3,Rodriguez High School,3109
4,Figueroa High School,2313
5,Huang High School,2299
6,Wilson High School,2129
7,Ford High School,2123
8,Cabrera High School,1744
9,Wright High School,1682


Merge school summary and reading_score totals for each school

In [23]:
school_sum_all = pd.merge(school_sum_all,school_sum_rp,on="school_name") # run only once
school_sum_all

,school_name,students_count,math_pass,reading_pass
0,Bailey High School,4976,3216,3946
1,Johnson High School,4761,3040,3727
2,Hernandez High School,4635,3001,3624
3,Rodriguez High School,3999,2562,3109
4,Figueroa High School,2949,1880,2313
5,Huang High School,2917,1847,2299
6,Ford High School,2739,1801,2123
7,Wilson High School,2283,2076,2129
8,Cabrera High School,1858,1664,1744
9,Wright High School,1800,1625,1682


Compute % Passing Math by school

In [24]:
school_sum_all["% math pass"] = school_sum_all["math_pass"]* 100/school_sum_all["students_count"]


Compute % Passing Reading by school

In [25]:
school_sum_all["% reading pass"] = school_sum_all["reading_pass"]* 100/school_sum_all["students_count"]



In [26]:
school_sum_all

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass
0,Bailey High School,4976,3216,3946,64.630225,79.300643
1,Johnson High School,4761,3040,3727,63.852132,78.281874
2,Hernandez High School,4635,3001,3624,64.746494,78.187702
3,Rodriguez High School,3999,2562,3109,64.066017,77.744436
4,Figueroa High School,2949,1880,2313,63.750424,78.433367
5,Huang High School,2917,1847,2299,63.318478,78.813850
6,Ford High School,2739,1801,2123,65.753925,77.510040
7,Wilson High School,2283,2076,2129,90.932983,93.254490
8,Cabrera High School,1858,1664,1744,89.558665,93.864370
9,Wright High School,1800,1625,1682,90.277778,93.444444


In [27]:
school_sum_all = pd.merge(school_sum_all,schools_df[["school_name","budget","type"]],on="school_name") #run only once

Compute budget per student

In [28]:
school_sum_all["budget per student"] = school_sum_all["budget"]/school_sum_all["students_count"]
# school_sum_all

In [29]:
school_sum_math = pd.DataFrame(schoolstu_df.groupby(["school_name"])["math_score"].sum().reset_index())
school_sum_math.columns = ["school_name", "math_total"]
# school_sum_math

In [30]:
school_sum_all = pd.merge(school_sum_all,school_sum_math,on="school_name")
# school_sum_all

In [31]:
school_sum_all["Average Math Score"] = school_sum_all["math_total"]/school_sum_all["students_count"]
# school_sum_all

In [32]:
school_sum_reading = pd.DataFrame(schoolstu_df.groupby(["school_name"])["reading_score"].sum().reset_index())
school_sum_reading.columns = ["school_name", "reading_total"]
# school_sum_reading

In [33]:
school_sum_all = pd.merge(school_sum_all,school_sum_reading,on="school_name")
# school_sum_all

In [34]:
school_sum_all["Average Reading Score"] = school_sum_all["reading_total"]/school_sum_all["students_count"]
# school_sum_all

In [35]:
school_sum_all["Overall passing rate"] = (school_sum_all["% math pass"] + school_sum_all["% reading pass"])/2
# school_sum_all.sort(on="Overall passing rate")

In [36]:
school_sum_all = school_sum_all.sort_values("Overall passing rate",ascending=False).reset_index(drop=True)

School Summary


Create an overview table that summarizes key metrics about each school, including:


School Name
School Type
Total Students
Total School Budget
Per Student Budget
Average Math Score
Average Reading Score
% Passing Math
% Passing Reading
Overall Passing Rate (Average of the above two)

In [37]:
school_sum_all

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student,math_total,Average Math Score,reading_total,Average Reading Score,Overall passing rate
0,Wilson High School,2283,2076,2129,90.932983,93.254490,1319574,Charter,578.0,190115,83.274201,191748,83.989488,92.093736
1,Pena High School,962,882,887,91.683992,92.203742,585858,Charter,609.0,80654,83.839917,80851,84.044699,91.943867
2,Wright High School,1800,1625,1682,90.277778,93.444444,1049400,Charter,583.0,150628,83.682222,151119,83.955000,91.861111
3,Cabrera High School,1858,1664,1744,89.558665,93.864370,1081356,Charter,582.0,154329,83.061895,156027,83.975780,91.711518
4,Holden High School,427,387,396,90.632319,92.740047,248087,Charter,581.0,35784,83.803279,35789,83.814988,91.686183
5,Thomas High School,1635,1475,1519,90.214067,92.905199,1043130,Charter,638.0,136389,83.418349,137093,83.848930,91.559633
6,Griffin High School,1468,1317,1371,89.713896,93.392371,917500,Charter,625.0,122360,83.351499,123043,83.816757,91.553134
7,Shelton High School,1761,1583,1631,89.892107,92.617831,1056600,Charter,600.0,146796,83.359455,147441,83.725724,91.254969
8,Bailey High School,4976,3216,3946,64.630225,79.300643,3124928,District,628.0,383393,77.048432,403225,81.033963,71.965434
9,Ford High School,2739,1801,2123,65.753925,77.510040,1763916,District,644.0,211184,77.102592,221164,80.746258,71.631982


List the top 5 schools based on 'Overall passing rate'

In [38]:
school_sum_all.head(5)

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student,math_total,Average Math Score,reading_total,Average Reading Score,Overall passing rate
0,Wilson High School,2283,2076,2129,90.932983,93.254490,1319574,Charter,578.0,190115,83.274201,191748,83.989488,92.093736
1,Pena High School,962,882,887,91.683992,92.203742,585858,Charter,609.0,80654,83.839917,80851,84.044699,91.943867
2,Wright High School,1800,1625,1682,90.277778,93.444444,1049400,Charter,583.0,150628,83.682222,151119,83.955000,91.861111
3,Cabrera High School,1858,1664,1744,89.558665,93.864370,1081356,Charter,582.0,154329,83.061895,156027,83.975780,91.711518
4,Holden High School,427,387,396,90.632319,92.740047,248087,Charter,581.0,35784,83.803279,35789,83.814988,91.686183


List the bottom 5 schools based on 'Overall passing rate'

In [39]:
school_sum_all = school_sum_all.sort_values("Overall passing rate",ascending=True).reset_index(drop=True)
school_sum_all.head(5)

,school_name,students_count,math_pass,reading_pass,% math pass,% reading pass,budget,type,budget per student,math_total,Average Math Score,reading_total,Average Reading Score,Overall passing rate
0,Rodriguez High School,3999,2562,3109,64.066017,77.744436,2547363,District,637.0,307294,76.842711,322898,80.744686,70.905226
1,Huang High School,2917,1847,2299,63.318478,78.813850,1910635,District,655.0,223528,76.629414,236810,81.182722,71.066164
2,Johnson High School,4761,3040,3727,63.852132,78.281874,3094650,District,650.0,366942,77.072464,385481,80.966394,71.067003
3,Figueroa High School,2949,1880,2313,63.750424,78.433367,1884411,District,639.0,226223,76.711767,239335,81.158020,71.091896
4,Hernandez High School,4635,3001,3624,64.746494,78.187702,3022020,District,652.0,358238,77.289752,375131,80.934412,71.467098


In [40]:
Math Scores by Grade


Table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

SyntaxError: invalid syntax (<ipython-input-40-b1fd335baec4>, line 1)

In [ ]:
school_grade_math_sum = pd.DataFrame(schoolstu_df.groupby(["school_name","grade"])["math_score"].mean().reset_index())
school_grade_math_sum.columns = ["school_name", "grade","math_score"]
school_grade_math_sum["dgrade"] = pd.to_numeric(school_grade_math_sum["grade"].str.extract('(^\d*)'))

school_grade_math_sum = school_grade_math_sum.sort_values(["school_name","dgrade"],ascending=True).reset_index(drop=True)
school_grade_math_sum = school_grade_math_sum.rename(columns={'math_score':'Average Math Score'})
school_grade_math_sum = school_grade_math_sum.drop(columns=["dgrade"])
school_grade_math_sum

Reading Scores by Grade


Table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [ ]:
school_grade_reading_sum = pd.DataFrame(schoolstu_df.groupby(["school_name","grade"])["reading_score"].mean().reset_index())
school_grade_reading_sum.columns = ["school_name", "grade","reading_score"]
school_grade_reading_sum["dgrade"] = pd.to_numeric(school_grade_reading_sum["grade"].str.extract('(^\d*)'))

school_grade_reading_sum = school_grade_reading_sum.sort_values(["school_name","dgrade"],ascending=True).reset_index(drop=True)
school_grade_reading_sum = school_grade_reading_sum.rename(columns={'reading_score':'Average Reading Score'})
school_grade_reading_sum = school_grade_reading_sum.drop(columns=["dgrade"])
school_grade_reading_sum

In [ ]:
bins = [560,600,620,640,680]

# Create the names for the four bins
group_names = ['Low', 'Medium', 'High', 'Very High']

In [ ]:
# Cut budget and place the scores into bins
pd.cut(school_sum_all["budget per student"], bins, labels=group_names)

In [ ]:
school_sum_all["Budget bin"]= pd.cut(
    school_sum_all["budget per student"], bins, labels=group_names)
# school_sum_all

Scores by School Spending


Table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:


Average Math Score
Average Reading Score
% Passing Math
% Passing Reading
Overall Passing Rate (Average of the above two)

In [ ]:
school_sum_all = school_sum_all.sort_values("Overall passing rate",ascending=False).reset_index(drop=True)
school_sum_all

In [ ]:
bins = [0,2000,4000,6000]

# Create the names for the four bins
group_names = ['Small', 'Medium', 'Large']

In [ ]:
# Cut school size and place the scores into bins
pd.cut(school_sum_all["students_count"], bins, labels=group_names)

Scores by School Size


Repeat of the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).

In [ ]:
school_sum_all["School size bin"]= pd.cut(
    school_sum_all["students_count"], bins, labels=group_names)
school_sum_all

School Type, Budget and Overall Passing Rate summary

In [ ]:
school_type_budget_sum = pd.DataFrame(school_sum_all.groupby(["type","Budget bin"])["Overall passing rate"].mean().reset_index())
school_type_budget_sum

School Type, School Size and Overall Passing Rate summary

In [ ]:
school_typ_size_sum= pd.DataFrame(school_sum_all.groupby(["type","School size bin"])["Overall passing rate"].mean().reset_index())
school_typ_size_sum

Three observable trends based on the data.
<li> Charter Schools fare well in Overall passing rate compared to District Schools </li>
<li> Smaller school size results in higher Overall passing rate </li>
<li> Budget per student is less in Charter Schools but still Overall passing rate is high compared to District Schools </li>